In [1]:
import sklearn
import numpy as np
import pandas as pd

import torch
cuda_available = torch.cuda.is_available()

from simpletransformers.ner import NERModel, NERArgs

from nervaluate import Evaluator

In [2]:
model_name='bert-base-multilingual-cased'

In [3]:
train_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\train.csv")
eval_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\val.csv")
test_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\test.csv")

In [4]:
label_list = [
    "O", 
    "B-PER", "I-PER",
    "B-ORG", "I-ORG",
    "B-LOC", "I-LOC",
    "B-PROD", "I-PROD",
    "B-WA", "I-WA",
    "B-EV", "I-EV",
]

In [5]:
# Create a NERModel
model_args=NERArgs()
# model_args.train_batch_size=64
# model_args.overwrite_output_dir=True
model_args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",

    "fp16": True,
    "fp16_opt_level": "O1",
    "max_seq_length": 512,
    "train_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    "num_train_epochs": 5,
    "weight_decay": 0,
    "learning_rate": 4e-5,

    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,

    "logging_steps": 50,
    "save_steps": 2000,

    "overwrite_output_dir": True,
    "reprocess_input_data": False,
    "evaluate_during_training": True,

    "do_lower_case": False, #True if using uncased models
}


In [6]:
# model = NERModel('bert', 
#                 'outputs/checkpoint-2000',
#                 args=model_args,
#                 labels=label_list,
#                 use_cuda=cuda_available,
#                 )

In [7]:
model = NERModel('bert',
                model_name,
                args=model_args,
                labels=label_list,
                use_cuda=cuda_available,
                )

model.train_model(train_df,  eval_data=eval_df)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

(2970,
 defaultdict(list,
             {'global_step': [594, 1188, 1782, 2000, 2376, 2970],
              'train_loss': [0.3238993287086487,
               0.11358702927827835,
               0.15126657485961914,
               0.02228463813662529,
               0.160630464553833,
               0.03641805425286293],
              'eval_loss': [0.21462016832708514,
               0.199173561578629,
               0.20158505086918305,
               0.20604518446258752,
               0.20865442137227663,
               0.22433117833988578],
              'precision': [0.6868029739776952,
               0.7102974828375286,
               0.7711319822046465,
               0.7245724572457246,
               0.7249219794917521,
               0.759981211836543],
              'recall': [0.6499560246262093,
               0.6824978012313104,
               0.6860158311345647,
               0.7080035180299032,
               0.7150395778364116,
               0.7115215479331575],
        

In [10]:
result, model_outputs, y_pred = model.eval_model(test_df)

2022-01-08 23:27:35 simpletransformers.ner.ner_model INFO:  Features loaded from cache at cache_dir/cached_dev_bert_512_13_1485
Running Evaluation: 100%|██████████| 186/186 [00:12<00:00, 14.87it/s]
2022-01-08 23:27:48 simpletransformers.ner.ner_model INFO: {'eval_loss': 0.10372366794192242, 'precision': 0.886421550782672, 'recall': 0.8522926146307316, 'f1_score': 0.8690221270521056}


In [11]:
result

{'eval_loss': 0.10372366794192242,
 'precision': 0.886421550782672,
 'recall': 0.8522926146307316,
 'f1_score': 0.8690221270521056}

In [12]:
df_x_test=test_df.groupby('sentence_id')['words'].apply(list)
x_test=list(df_x_test)

df_y_test=test_df.groupby('sentence_id')['labels'].apply(list)
y_test=list(df_y_test)

In [13]:
predictions, raw_outputs = model.predict(
    # ["@PolsekAlian PolriBerikan inovasi-inovasi Polri yang memudahkan masyarakat, kegiatan pemolisian masyarakat dan seluruh tindakan Kepolisian dalam menghadapi gangguan Kamtibmas, dan lain-lain - Jenderal Polisi Drs. Listyo Sigit Prabowo, https://t.co/f8GZ6VPqPD. - Kapolri #PolriTVRadioPresisi"],
    ["@ShopeeID Bismillah yok menang ���� SHOPEE @ShopeeID #44ShopeeMamamoo #ShopeexMAMAMOO #AdaMamamoodiShopee"], 
    ["@OPPOIndonesia ✍Butuh memori besar 128GB di OPPO A54 buat simpan file-file tugas kuliah dan testimoni online shop -ku ��Semoga oppo A54 bisa menunjang penghasilan keluargaku �� @OPPOIndonesia @nisa_rkt @Bebheey @kus_ica @Erna19_ @Saftriyuni #OPPOA54 https://t.co/bMYIiRb4QF �� 2.921"]
        )

2022-01-08 23:27:48 simpletransformers.ner.ner_model INFO:  Converting to features started.
Running Prediction: 100%|██████████| 1/1 [00:00<00:00, 26.66it/s]


In [14]:
predictions

[[{'@ShopeeID': 'O'},
  {'Bismillah': 'O'},
  {'yok': 'O'},
  {'menang': 'O'},
  {'����': 'O'},
  {'SHOPEE': 'B-PROD'},
  {'@ShopeeID': 'O'},
  {'#44ShopeeMamamoo': 'O'},
  {'#ShopeexMAMAMOO': 'O'},
  {'#AdaMamamoodiShopee': 'O'}]]

In [15]:
evaluator = Evaluator(y_test, y_pred, tags=['LOC', 'PER', 'ORG', 'PROD', 'WA', 'EV'], loader="list")

In [16]:
results, results_per_tag = evaluator.evaluate()

In [17]:
results['partial']

{'correct': 2495,
 'incorrect': 0,
 'partial': 81,
 'missed': 281,
 'spurious': 171,
 'possible': 2857,
 'actual': 2747,
 'precision': 0.9230069166363305,
 'recall': 0.8874693734686734,
 'f1': 0.9048893647394717}

In [18]:
results['exact']

{'correct': 2495,
 'incorrect': 81,
 'partial': 0,
 'missed': 281,
 'spurious': 171,
 'possible': 2857,
 'actual': 2747,
 'precision': 0.9082635602475427,
 'recall': 0.8732936646832342,
 'f1': 0.890435403283369}

In [19]:
import re

temp_l_l=list()
for i in range(len(y_test)):
    temp_l=list()
    print(x_test[i])
    print(len(x_test[i]))
    print("y_test       ",len(y_test[i]),y_test[i])
    print("y_pred       ",len(y_pred[i]),y_pred[i])
    for j in range(len(y_pred[i])):
        temp_l.append((x_test[i][j],y_test[i][j],y_pred[i][j]))
    temp_l_l.append(temp_l)
    print()

['@GyuuPotter', 'iyhh', ',', 'soalnya', 'mwu', 'kerjain', 'tugas', 'jg��']
8
y_test        8 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
y_pred        8 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

['@ShopeeID', 'Bismillah', 'yok', 'menang', '����', 'SHOPEE', '@ShopeeID', '#44ShopeeMamamoo', '#ShopeexMAMAMOO', '#AdaMamamoodiShopee']
10
y_test        10 ['O', 'O', 'O', 'O', 'O', 'B-PROD', 'O', 'O', 'O', 'O']
y_pred        10 ['O', 'O', 'O', 'O', 'O', 'B-PROD', 'O', 'O', 'O', 'O']

['RT', '@kukluxcats', ':', 'Selevel', 'gubernur', 'Jatim', 'aja', 'udah', 'maen', 'BuzzeRp', ',', 'emang', 'udah', 'parah', 'kayanya', 'kompetensi', 'pejabat', 'kita', 'sampe', 'sampe', 'apa', 'apa', 'musti', 'dipromote', 'BuzzeRp', 'https://t.co/kfpglITYzw']
26
y_test        26 ['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
y_pred        26 ['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [ ]:
with open("./results_pred.txt", "w") as f:
    str_results = repr(y_pred)
    f.write(str_results)

In [ ]:
with open("./results_test_pred.txt", "w",encoding="utf-8") as f:
    str_results = repr(temp_l_l)
    f.write(str_results)

In [17]:
with open("./results_1_batch.txt", "w") as f:
    str_results = repr(results)
    f.write(str_results)

In [18]:
with open("./results_per_tag_1_batch.txt", "w") as f:
    str_results_per_tag = repr(results_per_tag)
    f.write(str_results_per_tag)

In [19]:
# import re
# for i in range(len(y_test)):
#     if len(y_test[i]) != len(y_pred[i]):
#         print(x_test[i])
#         print(len(x_test[i]))
#         print("y_test       ",len(y_test[i]),y_test[i])
#         print("y_pred before",len(y_pred[i]),y_pred[i])
#         for j in range(len(x_test[i])):
#             # if '�' in x_test[i][j]: 
#             qmark=re.match(r'^�+$',x_test[i][j])
#             if qmark:
#                 print(qmark)
#                 y_pred[i].insert(j,'O')
#         print("y_pred after ",len(y_pred[i]), y_pred[i])
#         print()

In [20]:
import re

temp_l_l=list()
for i in range(len(y_test)):
    if y_test != y_pred:
        temp_l=list()
        print(x_test[i])
        print(len(x_test[i]))
        print("y_test       ",len(y_test[i]),y_test[i])
        print("y_pred before",len(y_pred[i]),y_pred[i])
        for j in range(len(x_test[i])):
            temp_l.append((x_test[i][j],y_test[i][j],y_pred[i][j]))
        temp_l_l.append(temp_l)
        print()

['@GyuuPotter', 'iyhh', ',', 'soalnya', 'mwu', 'kerjain', 'tugas', 'jg��']
8
y_test        8 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
y_pred before 8 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

['@ShopeeID', 'Bismillah', 'yok', 'menang', '����', 'SHOPEE', '@ShopeeID', '#44ShopeeMamamoo', '#ShopeexMAMAMOO', '#AdaMamamoodiShopee']
10
y_test        10 ['O', 'O', 'O', 'O', 'O', 'B-PROD', 'O', 'O', 'O', 'O']
y_pred before 10 ['O', 'O', 'O', 'O', 'O', 'B-PROD', 'O', 'O', 'O', 'O']

['RT', '@kukluxcats', ':', 'Selevel', 'gubernur', 'Jatim', 'aja', 'udah', 'maen', 'BuzzeRp', ',', 'emang', 'udah', 'parah', 'kayanya', 'kompetensi', 'pejabat', 'kita', 'sampe', 'sampe', 'apa', 'apa', 'musti', 'dipromote', 'BuzzeRp', 'https://t.co/kfpglITYzw']
26
y_test        26 ['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
y_pred before 26 ['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [21]:
temp_l_l

[[('@GyuuPotter', 'O', 'O'),
  ('iyhh', 'O', 'O'),
  (',', 'O', 'O'),
  ('soalnya', 'O', 'O'),
  ('mwu', 'O', 'O'),
  ('kerjain', 'O', 'O'),
  ('tugas', 'O', 'O'),
  ('jg��', 'O', 'O')],
 [('@ShopeeID', 'O', 'O'),
  ('Bismillah', 'O', 'O'),
  ('yok', 'O', 'O'),
  ('menang', 'O', 'O'),
  ('����', 'O', 'O'),
  ('SHOPEE', 'B-PROD', 'B-PROD'),
  ('@ShopeeID', 'O', 'O'),
  ('#44ShopeeMamamoo', 'O', 'O'),
  ('#ShopeexMAMAMOO', 'O', 'O'),
  ('#AdaMamamoodiShopee', 'O', 'O')],
 [('RT', 'O', 'O'),
  ('@kukluxcats', 'O', 'O'),
  (':', 'O', 'O'),
  ('Selevel', 'O', 'O'),
  ('gubernur', 'B-PER', 'B-PER'),
  ('Jatim', 'I-PER', 'I-PER'),
  ('aja', 'O', 'O'),
  ('udah', 'O', 'O'),
  ('maen', 'O', 'O'),
  ('BuzzeRp', 'O', 'O'),
  (',', 'O', 'O'),
  ('emang', 'O', 'O'),
  ('udah', 'O', 'O'),
  ('parah', 'O', 'O'),
  ('kayanya', 'O', 'O'),
  ('kompetensi', 'O', 'O'),
  ('pejabat', 'O', 'O'),
  ('kita', 'O', 'O'),
  ('sampe', 'O', 'O'),
  ('sampe', 'O', 'O'),
  ('apa', 'O', 'O'),
  ('apa', 'O', 'O'),
  (